In [26]:
import requests
from bs4 import BeautifulSoup
import sqlite3
import time

In [28]:
ORG_URL = "https://github.com/orgs/google/repositories"
BASE_URL = "https://github.com"

In [29]:
HEADERS = {
    "User-Agent": (
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/58.0.3029.110 Safari/537.3"
    )
}

In [30]:
# Google のリポジトリ一覧ページ取得
time.sleep(1)
res = requests.get(ORG_URL, headers=headers, timeout=10)
print("ステータスコード:", res.status_code)
res.raise_for_status()

soup = BeautifulSoup(res.text, "html.parser")

repo_links = []
seen = set()
name_language_stars = {}

# /google/xxx 形式のリンクだけを収集
for a in soup.find_all("a", href=True):
    href = a["href"]

    if href.startswith("/google/") and href.count("/") == 2:
        if href not in seen:
            seen.add(href)
            repo_links.append(a)

# 処理
for a in repo_links:
    name = a.get_text(strip=True)
    repo_path = a["href"]
    repo_url = BASE_URL + repo_path

    print(f"\n--- {name} を取得中: {repo_url}")
    time.sleep(1)

    r = requests.get(repo_url, headers=headers, timeout=10)
    if r.status_code != 200:
        print("  リポジトリページ取得失敗:", r.status_code)
        continue

    rsoup = BeautifulSoup(r.text, "html.parser")

    # 言語を抽出（例: "Python 53%" → Python）
    language = "Unknown"
    for tag in rsoup.find_all("a"):
        text = tag.get_text(strip=True)
        if text.endswith("%") and " " in text:
            language = text.split()[0]
            break
    lang_tag = rsoup.find("span", class_="color-fg-default text-bold mr-1")
    language = lang_tag.get_text(strip=True) if lang_tag else "Unknown"    

    # スター数を抽出（例: "1.2k stars" → 1.2k）
    stars = "0"
    for tag in rsoup.find_all("a"):
        text = tag.get_text(strip=True)
        if "stars" in text:
            stars = text.split()[0]
            break

    print("SCRAPED:", name, language, stars)
    name_language_stars[name] = (language, stars)

print("見つかったリポジトリ数:", len(repo_links))
print("件数:", len(name_language_stars))
print("例:", list(name_language_stars.items())[:3])

ステータスコード: 200

--- wasefire を取得中: https://github.com/google/wasefire
SCRAPED: wasefire Rust 129stars

--- orbax を取得中: https://github.com/google/orbax
SCRAPED: orbax Python 455stars

--- adk-python を取得中: https://github.com/google/adk-python
SCRAPED: adk-python Python 15.6kstars

--- osv-scalibr を取得中: https://github.com/google/osv-scalibr
SCRAPED: osv-scalibr Go 538stars

--- or-tools を取得中: https://github.com/google/or-tools
SCRAPED: or-tools C++ 12.7kstars

--- angle を取得中: https://github.com/google/angle
SCRAPED: angle C++ 3.8kstars

--- koladata を取得中: https://github.com/google/koladata
SCRAPED: koladata C++ 27stars

--- nomulus を取得中: https://github.com/google/nomulus
SCRAPED: nomulus Java 1.8kstars

--- perfetto を取得中: https://github.com/google/perfetto
SCRAPED: perfetto C++ 5kstars

--- nearby を取得中: https://github.com/google/nearby
SCRAPED: nearby C++ 888stars

--- skia-buildbot を取得中: https://github.com/google/skia-buildbot
SCRAPED: skia-buildbot Go 158stars

--- device-infra を取得中: htt

In [46]:
import sqlite3

path = ''
db_name = 'test.db'
db_path = path + db_name

In [47]:
try:
    conn = sqlite3.connect(db_path)
    cur = conn.cursor()

    sql = '''
        CREATE TABLE google (
            name TEXT,
            language TEXT,
            stars INT
        );
    '''

    cur.execute(sql)
    conn.commit()
    print("テーブル作成OK")

except sqlite3.Error as e:
    print(f"エラーが発生しました: {e}")

finally:
    conn.close()

テーブル作成OK


In [48]:
try:
    conn = sqlite3.connect(db_path)
    cur = conn.cursor()

    sql = "INSERT INTO google (name, language, stars) VALUES (?, ?, ?)"

    for name, (language, stars) in name_language_stars.items():
        cur.execute(sql, (name, language, stars))

    conn.commit()
    print("INSERT 完了")

except sqlite3.Error as e:
    print(f"エラーが発生しました: {e}")

finally:
    conn.close()

INSERT 完了


In [49]:
try:
    conn = sqlite3.connect(db_path)
    cur = conn.cursor()

    sql = "SELECT * FROM google"
    cur.execute(sql)

except sqlite3.Error as e:
    print(f"エラーが発生しました: {e}")

else:
    for idx, (name, language, stars) in enumerate(cur, start=1):
        print(idx, name, language, stars)

finally:
    conn.close()

1 wasefire Rust 129stars
2 orbax Python 455stars
3 adk-python Python 15.6kstars
4 osv-scalibr Go 538stars
5 or-tools C++ 12.7kstars
6 angle C++ 3.8kstars
7 koladata C++ 27stars
8 nomulus Java 1.8kstars
9 perfetto C++ 5kstars
10 nearby C++ 888stars
11 skia-buildbot Go 158stars
12 device-infra Java 58stars
13 selinux-policy-languages Unknown 14stars
14 aarch64-esr-decoder Rust 99stars
15 kotlin-fhirpath Kotlin 6stars
16 site-kit-wp JavaScript 1.3kstars
17 adk-go Go 4.9kstars
18 dawn C++ 781stars
19 yggdrasil-decision-forests C++ 622stars
20 XNNPACK C 2.2kstars
21 tunix Python 1.9kstars
22 dwh-migration-tools Java 54stars
23 desugar_jdk_libs Java 389stars
24 conscrypt Java 1.4kstars
25 tcmalloc C++ 5kstars
26 meridian Python 1.2kstars
27 filonov Python 11stars
28 xls C++ 1.4kstars
29 osv-scanner Go 8.1kstars
30 skia C++ 10.3kstars
